# Create a Terrace Defect on a Slab

Create a terrace defect by adding layers to a portion of a slab, defined by a cutting plane and number of additional layers.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Material (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Click "Run" > "Run All" to run all cells.
1. Wait for the run to complete.
1. Scroll down to view results.

## Notes

1. The input material must be a Slab, or slab will be generated with provided parameters.

## 1. Prepare the Environment
### 1.1. Set up the notebook
The cut direction, pivot coordinate, and number of added layers define the terrace defect, shown in the image below.

<img src="https://github.com/Exabyte-io/documentation/raw/88451ce38b0f57f804dcb5010a1a95bb9b9e50bc/images/notebooks/terrace_settings.webp" alt="Terrace parameters visualized" width="400"/>

In [ ]:
# Material selection
MATERIAL_INDEX = 0  # Which material to use from input list

# Terrace parameters
CUT_DIRECTION = [1, 0, 0]  # Normal vector describing a plane that cuts the terrace from added layers (Miller indices)
PIVOT_COORDINATE = [0.5, 0.5, 0.5]  # Point the cutting plane passes through, in crystal coordinates
NUMBER_OF_ADDED_LAYERS = 1  # Height of terrace in atomic layers
USE_CARTESIAN_COORDINATES = False  # Use cartesian instead of crystal coordinates
ROTATE_TO_MATCH_PBC = True  # Rotate to match periodic boundary conditions

# Slab parameters for creating a new slab if provided material is not a slab
DEFAULT_SLAB_PARAMETERS = {
    "miller_indices": (0, 0, 1),
    "thickness": 6,
    "vacuum": 10.0,
    "USE_ORTHOGONAL_C": True,
    "xy_supercell_matrix": [[5, 0], [0, 5]]
}

# Visualization parameters
SHOW_INTERMEDIATE_STEPS = True
CELL_REPETITIONS_FOR_VISUALIZATION = [3, 3, 1]  # Structure repeat in view

### 1.2. Install packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install`.

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("https://exabyte-io.github.io/api-examples/mat3ra_api_examples-0.1.dev1+gd196380-py3-none-any.whl", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("")

### 1.3. Load input material

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())
material = materials[MATERIAL_INDEX]

### 1.4. Create a slab if the input material is not a slab

In [ ]:
from mat3ra.made.tools.build.slab.helpers import create_slab_if_not, SlabConfiguration

default_slab_config = SlabConfiguration.from_parameters(
    material_or_dict=material,
    miller_indices=DEFAULT_SLAB_PARAMETERS["miller_indices"],
    number_of_layers=DEFAULT_SLAB_PARAMETERS["thickness"],
    vacuum=DEFAULT_SLAB_PARAMETERS["vacuum"],
)

slab = create_slab_if_not(material, default_slab_config)

### 1.5. Visualize slab

In [ ]:
from utils.visualize import visualize_materials

if SHOW_INTERMEDIATE_STEPS:
    print("Initial slab structure:")
    visualize_materials(slab, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION)
    visualize_materials(slab, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION, rotation="-90x")

## 2. Create target material
### 2.1. Set up terrace configuration and builder


In [ ]:
from mat3ra.made.tools.build.defect.terrace.helpers import create_terrace

terrace_slab = create_terrace(
    slab=slab,
    cut_direction=CUT_DIRECTION,
    pivot_coordinate=PIVOT_COORDINATE,
    number_of_added_layers=NUMBER_OF_ADDED_LAYERS,
    use_cartesian_coordinates=USE_CARTESIAN_COORDINATES,
    rotate_to_match_pbc=ROTATE_TO_MATCH_PBC
)


## 2.2. Print the parameters of the created terrace defect

In [ ]:

print("\nTerrace defect created with:")
print(f"Cut direction: {CUT_DIRECTION}")
print(f"Pivot point: {PIVOT_COORDINATE}")
print(f"Added layers: {NUMBER_OF_ADDED_LAYERS}")
print(f"Number of atoms: {len(terrace_slab.basis.elements.ids)}")

## 3. Visualize the result

In [ ]:
print("Final structure with terrace:")
visualize_materials(terrace_slab, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION)
visualize_materials(terrace_slab, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION, rotation="-90x")

## 4. Pass data to the outside runtime


In [ ]:
from utils.jupyterlite import set_materials

set_materials(terrace_slab)